In [2]:
code = """
%%sql
SELECT * FROM customers
LIMIT 1000
"""

In [6]:
import os
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

##################################
# Params
##################################

# Set this to your dbt service account token.
# Instructions for creating one of these is here:
# https://docs.getdbt.com/docs/dbt-cloud-apis/service-tokens#generating-service-account-tokens
token = os.environ['DBT_TOKEN']
# Set this to your dbt jobId by viewing the job url in your browser. 
# In https://cloud.getdbt.com/#/accounts/1/projects/665/jobs/917/, 
# 917 is the job id
jobId = 170565 

##################################
# Code
##################################

transport = AIOHTTPTransport(
  url="https://metadata.cloud.getdbt.com/graphql", 
  headers={'Authorization': f'Bearer {token}'}
)

async def main():
  async with Client(
    transport=transport, 
    fetch_schema_from_transport=True
  ) as session:

    query = gql(f"""
     {{
      models(jobId:{jobId}) {{
        name
        runGeneratedAt
        description
        columns {{
          name
          description
        }}
      }}
    }}
    """
    )
    result = await session.execute(query)
    return result

def buildModelBlock(model):
  def buildColumns(model):
    return "".join([buildColumnBlock(column) for column in model['columns']])
  def buildColumnBlock(col):
    nl = '\n'
    return f"""
    Name: {col['name']}{f"{nl}    Description: {col['description']}" if col['description'] else ""}
    """
  return f"""
  Model: {model['name']}
  Description: {model['description']}
  Generated At: {model['runGeneratedAt']}
  Columns: {buildColumns(model)}
  """ 

models = await main()
models['models']
'\n\n'.join([buildModelBlock(model) for model in models['models'] if model['name'] in code])

'\n  Model: customers\n  Description: One record per customer\n  Generated At: 2022-12-05T19:11:58.005Z\n  Columns: \n    Name: customer_id\n    Description: Primary key\n    \n    Name: first_name\n    \n    Name: last_name\n    \n    Name: first_order_date\n    Description: NULL when a customer has not yet placed an order.\n    \n    Name: most_recent_order_date\n    \n    Name: number_of_orders\n    \n  '